<a href="https://colab.research.google.com/github/giuliaulpiani/SWAQ/blob/main/SWAQ_QC_CODE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is the code to produce the two quality-controlled SWAQ datafiles (csv format) as in the TERN repository:

https://portal.tern.org.au/schools-weather-air-sydney-nsw/22077

Cite as:

Hart, M. , Maharaj, A. , Di Virgilio, G. , Ulpiani, G. (2021): Schools Weather and Air Quality (SWAQ) – Quality Controlled Urban Dataset – Sydney (NSW). Version 1.0.0. Terrestrial Ecosystem Research Network (TERN). Dataset. https://doi.org/10.5281/zenodo.5016296

It is written in very plain Python language for use also by beginners in Google Colab, a collaborative environment that runs in Google Drive.
When using the code:
- create a folder in Drive containing the code itself and the dummy_swaq_data.csv dataset. This dataset contains approximately 2-month data in the original format retrieved from the SWAQ Cloud, prior to time alignment; 
- make sure to update the project_folder in the "Import packages" section (first section) accordingly.

In addition to “YYYY-MM-DD_Raw.csv” where flags supplement but do not alter the original data and “YYYY-MM-DD_Cleaned.csv” that contains a ready-to-use cleaned dataset, the code generates the "swaq_data_aligned&resampled.xlsx" Excel file with the original dataset aligned in time (no quality control).

NB: to avoid any issues, please do not alter any names in the code.

In [4]:
# Import packages
! pip install mpu
! pip install XlsxWriter
import os
import pandas as pd 
import datetime
from calendar import monthrange
import numpy as np
from scipy import stats
import seaborn as sns
import mpu
import math
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from google.colab import drive
drive.mount('/content/gdrive')
project_folder = "/content/gdrive/My Drive/Colab Notebooks/SWAQ/PUBLIC CODE/"
os.chdir(project_folder)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# ALIGN STATIONS IN TIME AND RESAMPLE TO AVOID GAPS IN TIME

In [5]:
# function to verify if there is any gap in the timeline. It returns "check passed" if no time misalignment is recorded
# and "WARNING: MISSING DATA" if the timeline is broken
def checkmissing(df):
    df['Time'] = df.index
    # calculate the time difference row by row and add it as a column
    df['deltat'] = (df['Time']-df['Time'].shift()).fillna(pd.Timedelta('0 days'))
    # express the difference in minutes and add it as a column
    df['ans'] = df['deltat'].apply(lambda x: x  / np.timedelta64(1,'m')).astype('int64') % (24*60)
    # print the count of unique values to see how many where different from 20 ± 1 minutes (sampling rate)
    count=(df['ans'].value_counts())
    print (count)
    okvalues=[0,19,20,21]
    total=0
    for z in okvalues:
        if z in count.keys():
            total+=count[z]
    totalok=total/len(df)*100
    if totalok==100:
        str='check passed'
        test=0
    else:
        str='WARNING: MISSING DATA'
        test=1
    print(str)
    df.drop(columns=['Time', 'deltat','ans'],inplace=True)
    return test

In [6]:
# import SWAQ datafile 
fpath = project_folder+'dummy_swaq_data.csv'
data = pd.read_csv(fpath,index_col=[0],parse_dates=True,dayfirst=True)

In [7]:
# Use conventional names for columns
oldnames=["AirTemp","AirHum","AirPres","WindDir","WindSpeed"]
newnames=["T","RH","p","wd","ws"]
for scroll in range(len(oldnames)):
    data.columns = data.columns.str.replace(oldnames[scroll], newnames[scroll])

In [8]:
# Get stations names by looking for unique elements in column name
stations = [col[:col.index("_")] for col in data.columns]
stations=np.asarray(stations)
stations=np.unique(stations)
# Split the SWAQ locations between those measuring meteo only (metonly) and those measuring air quality too (metoaq)
metonly=['DULW','KELL','NARE','TARE','NEWT']
# Get the ramaining locations by subtraction
listations=stations.tolist()
metoaq=list(set(listations)-set(metonly))

In [9]:
# Select the station having the longest time recording and insert it at the beginning of the list of stations. The timestamp of this station will be used to time-align the others.
first_st=data.iloc[-1,:].first_valid_index().split('_')[0]
listations.remove(first_st)
listations.insert(0, first_st)

In [11]:
# Append all subdatasets related to each station into a list, set the Timestamp as datetime and then as index. 
# Use "concat" to merge all dataset along the horizontal axis (axis=1) to align them in time.
mylist=[]
for i in range(len(listations)):
  subdata = data.filter(regex=listations[i])
  timei=(listations[i]+"_Timestamp")
  subdata[timei] = pd.to_datetime(subdata[timei],dayfirst=True)
  subdata = subdata.set_index(subdata[timei])
  subdata = subdata.loc[~subdata.index.duplicated(keep='first')]
  mylist.append(subdata)           
swdataor=pd.concat(mylist,axis=1)
swdataor['Time'] = pd.to_datetime(swdataor.index)
swdataor = swdataor.set_index(pd.DatetimeIndex(swdataor['Time']))

In [12]:
# Check time gaps
test=checkmissing(swdataor)

20    8980
5        5
15       1
0        1
Name: ans, dtype: int64


In [13]:
# If time gaps, apply resampling on Time column with frequency equal to measurement time step (20 mins). 
# If still gaps send a warning
swdata=swdataor.copy() 
if test==1:
    swdata['Time'] = swdata.index
    swdata=swdata.resample('20Min', on='Time').first()\
           .drop('Time', 1)
    test=checkmissing(swdata)
    if test==1:
        print('WARNING: THE DATA IS STILL SHOWING TIME GAPS. MANUAL CHECK NEEDED')

20    8982
0        1
Name: ans, dtype: int64
check passed


In [18]:
# write Excel File  
swdata = swdata[swdata.columns.drop(list(swdata.filter(regex='Timestamp')))]
filename=fpath.split('.')[0][-9:-1]+fpath.split('.')[0][-1]
out_path = project_folder+filename+"_aligned&resampled.xlsx"
writer = pd.ExcelWriter(out_path , engine='xlsxwriter')
swdata.to_excel(writer, sheet_name='Sheet1')
writer.save()

# QUALITY CONTROL & FLAGGING

In [19]:
# Recall aligned and resampled dataset if necessary
datapath = project_folder+filename+"_aligned&resampled.xlsx"
swdataor = pd.read_excel(datapath)
# Set timestamp as dataframe index
swdataor = swdataor.set_index(['Time'])
swdataor.head(3)

,BROO_NO2,BROO_SO2,BROO_CO,BROO_O3,BROO_PM25,BROO_PM10,BROO_T,BROO_RH,BROO_p,BROO_Rain,BROO_wd,BROO_ws,DULW_T,DULW_RH,DULW_p,DULW_Rain,DULW_wd,DULW_ws,DULW_Solarrad,GLEN_NO2,GLEN_SO2,GLEN_CO,GLEN_O3,GLEN_PM25,GLEN_PM10,GLEN_T,GLEN_RH,GLEN_p,GLEN_Rain,GLEN_wd,GLEN_ws,KELL_T,KELL_RH,KELL_p,KELL_Rain,KELL_wd,KELL_ws,KURN_NO2,KURN_SO2,KURN_CO,...,NARE_p,NARE_Rain,NARE_wd,NARE_ws,NEWT_T,NEWT_RH,NEWT_p,NEWT_Rain,NEWT_wd,NEWT_ws,OEHS_NO2,OEHS_SO2,OEHS_CO,OEHS_O3,OEHS_PM25,OEHS_PM10,OEHS_T,OEHS_RH,OEHS_p,OEHS_Rain,OEHS_wd,OEHS_ws,TARE_T,TARE_RH,TARE_p,TARE_Rain,TARE_wd,TARE_ws,UNSW_NO2,UNSW_SO2,UNSW_CO,UNSW_O3,UNSW_PM25,UNSW_PM10,UNSW_T,UNSW_RH,UNSW_p,UNSW_Rain,UNSW_wd,UNSW_ws
Time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-09-01 00:00:00,0.023,0.025,0.063,-0.001,2.5,4.4,18.6,58.5,1018.3,0.0,205.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.015,0.028,0.051,0.015,1.5,3.0,16.1,63.0,1001.3,0.0,338.0,0.6,NaN,NaN,NaN,NaN,NaN,NaN,0.032,0.033,0.092,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.012,0.184,0.046,-0.001,3.5,7.6,17.0,56.4,1016.8,0.0,347.0,0.7,NaN,NaN,NaN,NaN,NaN,NaN,0.021,0.023,0.097,0.012,3.4,8.5,17.5,56.8,1015.1,0.0,262.0,0.6
2019-09-01 00:20:00,0.023,0.021,0.029,-0.001,2.9,4.5,18.6,60.4,1018.2,0.0,356.0,0.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.010,0.028,0.055,0.020,1.7,3.1,16.6,58.6,1001.1,0.0,335.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.024,0.030,0.087,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.009,0.262,0.053,0.002,3.6,9.3,18.0,53.7,1016.5,0.0,60.0,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.021,0.022,0.107,0.014,3.3,8.6,18.0,55.8,1014.9,0.0,270.0,1.0
2019-09-01 00:40:00,0.018,0.007,0.048,0.000,3.2,4.5,19.5,55.2,1018.0,0.0,324.0,0.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.005,0.026,0.035,0.020,1.5,3.4,18.0,56.0,1000.9,0.0,354.0,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.021,0.028,0.078,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.013,0.357,0.043,-0.001,3.9,9.0,19.0,50.3,1016.4,0.0,2.0,1.9,NaN,NaN,NaN,NaN,NaN,NaN,0.022,0.017,0.080,0.017,3.4,10.2,18.4,53.4,1014.8,0.0,44.0,0.6


In [20]:
# Keep track of original dataset and initialize cleaned dataset
swdata = swdataor.copy()
sw_clean = swdataor.copy()

In [21]:
# Correct NEWT_wd by 180 degrees if before 26 Mar 2021, 11:30 am
mask = swdata.index <= '2021-03-26 11:30:00'
swdata.loc[mask,'NEWT_wd'] = np.where(swdata.loc[mask,'NEWT_wd']+180.0 > 360, swdata.loc[mask,'NEWT_wd']-180.0, swdata.loc[mask,'NEWT_wd']+180.0)
sw_clean.loc[mask,'NEWT_wd'] = np.where(sw_clean.loc[mask,'NEWT_wd']+180.0 > 360, sw_clean.loc[mask,'NEWT_wd']-180.0, sw_clean.loc[mask,'NEWT_wd']+180.0)
# Correct all RH measurements by applying a positive offset of (100-94.7), 
# where 94.7 is the absolute maximum measured on the 2019-31Jan2021 dataset
swdata.loc[:,swdata.filter(regex='_RH').columns]=swdata.loc[:,swdata.filter(regex='_RH').columns]+100-94.7
sw_clean.loc[:,sw_clean.filter(regex='_RH').columns]=sw_clean.loc[:,sw_clean.filter(regex='_RH').columns]+100-94.7
# Remove any column containing "Solarrad" (unknown measurement in Dulwich Hill)
swdata=swdata[swdata.columns.drop(list(swdata.filter(regex='Solarrad')))]
sw_clean=sw_clean[sw_clean.columns.drop(list(sw_clean.filter(regex='Solarrad')))]

In [22]:
# Create a flag column for each measured parameter
columns=swdata.columns
for col in columns:
    label=col+'_Flags'
    swdata[label]=['' for i in range(swdata.shape[0])]

In [23]:
##### CONTINUITY TEST: Flag missing rows
##### Set flag columns to "STF4.1" if data is missing
for col in columns:
  varname=col
  flagname=col+'_Flags'
  for ind in swdata.index:
      if pd.isnull(swdata.loc[ind,varname]):
          swdata.loc[ind,flagname] = 'STF4.1;'

In [24]:
##### FIXED RANGE TESTS: Remove non-physical values
meteovar=['T','RH','p','Rain','ws','wd'];oaqvar=['NO2','SO2','CO','O3','PM25','PM10']
met_lowerbound=[-52.0,0.0,500.0,0.0,0.0,0];oaq_lowerbound=[0.000,0.000,0.0000,0.000,0.000,0.000]
met_upperbound=[60.0,100,1100.0,200,60.0,360];oaq_upperbound=[2000/1000,2000/1000,10000/1000,2000/1000,2000,3276] # 5000 replaced by 3276 after perusing dataset
for st in stations: 
    for var in range(len(meteovar)): 
        varname=st+'_'+meteovar[var]
        flagname=st+'_'+meteovar[var]+'_Flags'
        indices_lo=list(np.where(swdata[varname] < met_lowerbound[var])[0])
        indices_up=list(np.where(swdata[varname] > met_upperbound[var])[0])
        indices=np.concatenate((indices_lo, indices_up)).astype(int)
        swdata.iloc[indices,swdata.columns.get_loc(flagname)] = swdata.iloc[indices,swdata.columns.get_loc(flagname)]+'STF2.1;'
        if meteovar[var] in ['RH','Rain','ws','wd']:
            sw_clean.iloc[indices_lo,sw_clean.columns.get_loc(varname)] = met_lowerbound[var]
        if meteovar[var] in ['RH','wd']:
            sw_clean.iloc[indices_up,sw_clean.columns.get_loc(varname)] = met_upperbound[var]
        del indices_up; del indices_lo; del indices
for st in metoaq:        
    for var in range(len(oaqvar)):
        varname=st+'_'+oaqvar[var]
        flagname=st+'_'+oaqvar[var]+'_Flags'
        indices_lo=list(np.where(swdata[varname] < oaq_lowerbound[var])[0])
        indices_up=list(np.where(swdata[varname] > oaq_upperbound[var])[0])
        indices=np.concatenate((indices_lo, indices_up)).astype(int)
        swdata.iloc[indices,swdata.columns.get_loc(flagname)] = swdata.iloc[indices,swdata.columns.get_loc(flagname)]+'STF2.1;'
        sw_clean.iloc[indices_lo,sw_clean.columns.get_loc(varname)] = oaq_lowerbound[var]
        sw_clean.iloc[indices_up,sw_clean.columns.get_loc(varname)] = np.nan

In [25]:
##### INTERNAL CONSISTENCY TESTS: Remove inconsistent values as for inter-parameter associations
for st in stations: 
    varname1=st+'_ws';varname2=st+'_wd'
    flagname1=st+'_ws_Flags';flagname2=st+'_wd_Flags'
    indices=list(np.where((swdata[varname1] == 0) & (swdata[varname2] != 0))[0])
    sw_clean.iloc[indices,sw_clean.columns.get_loc(varname1)] = np.nan
    sw_clean.iloc[indices,sw_clean.columns.get_loc(varname2)] = np.nan
    swdata.iloc[indices,swdata.columns.get_loc(flagname)] = swdata.iloc[indices,swdata.columns.get_loc(flagname)]+'STF2.2;'

In [26]:
##### PERSISTENCE TESTS according to Meek and Hatfield (1994): flag consecutive identical readings over 3 hours
# check if preceding value is the same using .shift() function
for col in columns:
    varname=col[0:13]
    flagname=col[0:13]+'_Flags'
    swdata['same_as_shift'] = swdata[varname].shift() != swdata[varname] 
    for name, group in swdata.groupby(swdata.same_as_shift.cumsum()):
        if (len(group) > 9) & (not ("Rain") in varname):
            swdata.loc[group.index,flagname] = swdata.loc[group.index,flagname] + 'STF3;'
            sw_clean.loc[group.index,varname] = np.nan
    swdata = swdata.drop(columns=['same_as_shift'])

In [27]:
##### DYNAMIC RANGE TESTS (site-specific extremes): for each station and parameter, verify if 
##### the measured value is an outlier with respect to the monthly dataset of all stations 
#####(Outlier definition: <p25-1.5IQR or >p75+1.5IQR)
# NB: d is a dictionary of outliers. The outmost key is a progressive index for each monthly period:
# By printing d[1] one can access all outliers pertaining to the first month of data, broken down by parameter
allvar = meteovar + oaqvar
allvar=[x for x in allvar if x not in ['Rain','RH','wd']]
allvar = ["_" + par for par in allvar]
idx=1
d = {}
for name, group in swdata.groupby(pd.Grouper(freq="M")):
    d[idx] = {}
    percent=len(group)/(monthrange(name.year, name.month)[1]*24*3)*100
    if percent < 90:
        swdata.loc[group.index,swdata.filter(regex='Flags').columns]=swdata.loc[group.index,swdata.filter(regex='Flags').columns]+"STF4.2;"
        sw_clean.loc[group.index,:]=np.nan
    else:
        for var in allvar:
            d[idx]['Period'] = str(name.month)+'/'+str(name.year)
            d[idx][var[1:]]= {}
            # Find the 75th and 25th percentile of the parameter in object (var) by looking at all stations
            var_cols = [col for col in swdata.columns if (var in col) & (not ("Flags") in col)]
            # Calculate outliers in cleaned dataset (OR remove previous flags???)
            q75,q25 = np.nanpercentile(sw_clean.loc[group.index,var_cols],[75,25])
            # Calculate the interquartile range
            intr_qr = q75-q25
            # Apply the definition of outlier to define the thresholds (max and min)
            max = q75+(1.5*intr_qr)
            min = q25-(1.5*intr_qr)   
            d[idx][var[1:]]['Upper'] = round(max,4)
            d[idx][var[1:]]['Lower'] = round(min,4)
            for i in var_cols:
                flagname=i+'_Flags'
                if idx==1:
                    indices_up=list(np.where(swdata.loc[group.index,i] > max)[0])
                    indices_lo=list(np.where(swdata.loc[group.index,i] < min)[0])
                else:
                    indices_up=list(np.where(swdata.loc[group.index,i] > max)[0])+np.argwhere(swdata.index < group.index[0]).flatten()[-1]+1
                    indices_lo=list(np.where(swdata.loc[group.index,i] < min)[0])+np.argwhere(swdata.index < group.index[0]).flatten()[-1]+1
                indices=np.concatenate((indices_up, indices_lo)).astype(int)
                swdata.iloc[indices,swdata.columns.get_loc(flagname)] = swdata.iloc[indices,swdata.columns.get_loc(flagname)]+'STF1.1;'
                del indices_up; del indices_lo; del indices
    idx+=1

In [29]:
##### STEP TESTS (site-specific extremes): for each station and parameter, verify if 
##### the step from previous measurement is an outlier with respect to the steps of all stations across the same month
#####(Wider outlier definition: <p25-3IQR or >p75+3IQR)
# NB: ds is a dictionary of outliers. The outmost key is a progressive index for each monthly period:
# By printing d[1] one can access all outliers pertaining to the first month of data, broken down by parameter

for col in columns:
    label=col+'_Delta'
    swdata[label]=abs(swdata[col] - swdata[col].shift(1))
    sw_clean[label]=abs(sw_clean[col] - sw_clean[col].shift(1))    
    
allvar = meteovar + oaqvar
allvar=[x for x in allvar if x not in ['Rain','wd']]
allvar = ["_" + par for par in allvar]
idx=1
ds = {}
for name, group in swdata.groupby(pd.Grouper(freq="M")):
    ds[idx] = {}
    percent=len(group)/(monthrange(name.year, name.month)[1]*24*3)*100
    if percent >= 90:
        for var in allvar:
            ds[idx]['Period'] = str(name.month)+'/'+str(name.year)
            ds[idx][var[1:]]= {}
            # Find the 75th and 25th percentile of the parameter in object (var) by looking at all stations
            var_cols = [col for col in swdata.columns if (var in col) & (("Delta") in col) & (not ("Flags") in col)]
            # Calculate outliers in cleaned dataset (OR remove previous flags???)
            q75,q25 = np.nanpercentile(sw_clean.loc[group.index,var_cols],[75,25])
            # Calculate the interquartile range
            intr_qr = q75-q25
            # Apply the definition of outlier to define the thresholds (max)
            max = q75+(3*intr_qr)  
            ds[idx][var[1:]] = round(max,4)
            for i in var_cols:
                flagname=i.replace('_Delta','')+'_Flags'
                if idx==1:
                    indices=list(np.where(swdata.loc[group.index,i] > max)[0])
                else:
                    indices=list(np.where(swdata.loc[group.index,i] > max)[0])+np.argwhere(swdata.index < group.index[0]).flatten()[-1]+1
                # Shall include line to check if indices in not null?
                if len(indices)==0:
                  continue
                else:
                  swdata.iloc[indices,swdata.columns.get_loc(flagname)] = swdata.iloc[indices,swdata.columns.get_loc(flagname)]+'STF1.2;'
                del indices
    idx+=1
swdata = swdata[swdata.columns.drop(list(swdata.filter(regex='Delta')))]
sw_clean = sw_clean[sw_clean.columns.drop(list(sw_clean.filter(regex='Delta')))]

In [32]:
##### APPLY COMBINATORIAL FLAGS and set data to NaN in cleaned dataset only if both dynamic range test and step test are simultaneously failed
##### NB: allow even more than 1 hour to complete, depending on the size of the dataset. For the dummy_swaq_data.csv file this section would take approx. 15 minutes.
for col in columns:
    varname=col[0:13]
    flagname=col[0:13]+'_Flags'
    for ind in swdata.index:
        if swdata.loc[ind,flagname]=='':
            swdata.loc[ind,flagname]=swdata.loc[ind,flagname]+'STF0;CF0'
        elif 'STF1.1;STF1.2;' in swdata.loc[ind,flagname]:# the "contains" function is used otherwise cells that have failed also other tests would not be removed
            swdata.loc[ind,flagname]=swdata.loc[ind,flagname]+'CF1;'
            sw_clean.loc[ind,varname] = np.nan
        elif ('STF2.1;' in swdata.loc[ind,flagname]) | ('STF2.2;' in swdata.loc[ind,flagname]):
            swdata.loc[ind,flagname]=swdata.loc[ind,flagname]+'CF2'
        elif swdata.loc[ind,flagname]=='STF3':
            swdata.loc[ind,flagname]=swdata.loc[ind,flagname]+'CF3'
        elif ('STF4.1;' in swdata.loc[ind,flagname]) | ('STF4.2;' in swdata.loc[ind,flagname]):
            swdata.loc[ind,flagname]=swdata.loc[ind,flagname]+'CF4'

In [33]:
# write csv files with datetime format to ISO 8601
swd=swdata; sw_c=sw_clean
swd.index = swd.index.to_series().apply(datetime.datetime.isoformat)
sw_c.index = sw_c.index.to_series().apply(datetime.datetime.isoformat)
# Get last readings' timestamp and convert it into isoformat then build up filename
filename1=swdata.index[-1].split('T')[0]+"_Raw.csv"
filename2=swdata.index[-1].split('T')[0]+"_Cleaned.csv"
swd.to_csv(filename1)
sw_c.to_csv(filename2)

END OF CODE